# Parameter Efficient Fine-tuning (QLoRA) de LLAMA 2

<div style="background-color:#D9EEFF;color:black;padding:2%;">
<h2>Enunciado del caso práctico</h2>

En este caso práctico, se propone al alumno la realización de Parameter Efficient Fine-tuning (PEFT) sobre uno de los LLMs más potentes y populares de la actualidad, su nombre es [LLAMA 2](https://ai.meta.com/llama/) de la compañía Meta (anteriormente Facebook).

Para este ejercicio concreto se propone el uso de [LLAMA-2-7b-chat](https://huggingface.co/NousResearch/Llama-2-7b-chat-hf) que es una versión de LLAMA-2 que tiene 7.000 millones de parámetros.

Concretamente, se propone un escenario en el que un periódico digital quiere generar automáticamente el título de sus artículos, la descripción y las palabras clave utilizadas para el SEO.

Un factor importante es que la generación de esta información no debe ser genérica, debe adaptarse al estilo y tipo de títulos, descripciones y palabras clave que ha generado el periódico para otros artículos pasados.

</div>

# Resolución del caso práctico

## 0. Instalación de librerías externas

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

## 1. Comportamiento de [LLAMA-2-7B-Chat](https://ai.meta.com/llama/) sin Fine-tuning

### 1.1. Lectura del modelo y tokenizador

En este ejercicio vamos a cargar el modelo de una manera especial. Concretamente, vamos a aplicar una técnica denominada QLoRA (Quantized LoRA).

QLoRA es un enfoque de ajuste eficiente que reduce el uso de memoria lo suficiente como para ajustar un modelo de 65B parámetros en una sola GPU de 48 GB, al tiempo que conserva el rendimiento completo de la tarea de ajuste de 16 bits.

Un modelo cuantificado es un modelo que tiene sus parámetros/pesos en un tipo de datos inferior al tipo de datos en el que fue entrenado. Por ejemplo, si se entrena con un tipo de dato float de 32 bits y luego se convierten esos parámetros/pesos a un tipo de datos inferior, como float de 16/8/4 bits, el efecto sobre el rendimiento del modelo es mínimo o nulo.

En este caso vamos a reducir la precisión de los parámetros del LLM a 4 bits. Esto ayuda a disminuir enormemente el consumo de memoria del modelo, lo que facilita su manipulación y ajuste (finetuning) en hardware más accesible, como una única GPU.

Repositorio GitHub: https://github.com/artidoro/qlora

Artículo: https://arxiv.org/abs/2305.14314

In [2]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
import torch

# Definimos los paramétros para bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

In [3]:
# Leemos el modelo pre-entrenado el modelo LLAMA2-7b-chat
model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Llama-2-7b-chat-hf",
    quantization_config=bnb_config,
    device_map={"": 0},
    # low_cpu_mem_usage=True # Reduccion del consumo de cpu y memoria al leer el modelo
)

model.config.use_cache = False
model.config.pretraining_tp = 1 # Un valor distinto de 1 activará el cálculo más preciso pero más lento de las capas lineales

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [4]:
from transformers import AutoTokenizer

# Leemos el tokenizador
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

### 1.2. Generación de texto

In [5]:
from transformers import pipeline

# Creamos un pipeline para la tokenización y generación del texto
llama2_pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)

In [6]:
prompt = """<<SYS>>
Comportate como un ChatBot amigable experto en programación en Python
<</SYS>>

Desarrolla un programa en Python que ordene una lista de 10 numeros enteros."""

In [7]:
prompt = "Actúa como si fueses el mayor experto en historia del mundo. Describe \
en pocas palabras lo que ocurrió en la segunda guerra mundial."

In [8]:
prompt_template = f"<s>[INST] {prompt} [/INST]"

# Invocamos el pipeline para realizar generación de texto
output = llama2_pipe(prompt_template)
print(output[0]['generated_text'].replace("[/INST]", "[/INST]\n\n"))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>[INST] Actúa como si fueses el mayor experto en historia del mundo. Describe en pocas palabras lo que ocurrió en la segunda guerra mundial. [/INST]

  ¡Ah, la Segunda Guerra Mundial! ¡Qué gran tema para un experto en historia como yo! ¡Buenos días!

En resumen, la Segunda Guerra Mundial fue un conflicto global que duró de 1939 a 1945 y enfrentó a las potencias del Eje (Alemania, Italia y Japón) contra las potencias aliadas (Estados Unidos, Reino Unido, Francia y la Unión Soviética).

La guerra comenzó en 1939 cuando Alemania, bajo el liderazgo de Adolf Hitler, invadió Polonia. La invasión fue seguida de una declaración de guerra por parte de Francia y Reino Unido, lo que llevó a una escalada de hostilidades en Europa y Asia.

La guerra en Europa se libró principalmente en dos frentes: el frente occidental, donde las fuerzas aliadas lucharon contra las fuerzas alemanas en Bélgica, Francia y Holanda; y el frente oriental, donde las fuerzas soviéticas lucharon contra las fuerzas aleman

In [9]:
prompt = """"La Revolución Industrial, que tuvo lugar principalmente en el siglo XIX, \
fue un período de grandes cambios tecnológicos, culturales y socioeconómicos que \
transformó a las sociedades agrarias en sociedades industriales. Durante este tiempo, \
hubo un cambio masivo de mano de obra de las granjas a las fábricas. Esto se debió a \
la invención de nuevas máquinas que podían realizar tareas más rápido y eficientemente \
que los humanos o los animales. Esta transición llevó a un aumento en la producción de \
bienes, pero también tuvo consecuencias negativas, como la explotación \laboral y la \
contaminación ambiental."""

In [10]:
prompt_template = f"<s>[INST] <<SYS>>Dado un artículo de noticias, proporciona \
los siguientes campos en un diccionario JSON: \'titulo\', \'SEO\' y \'resumen\'\
<</SYS>> {prompt} [/INST]"

# Invocamos el pipeline para realizar generación de texto
output = llama2_pipe(prompt_template)
print(output[0]['generated_text'].replace("[/INST]", "[/INST]\n\n"))

<s>[INST] <<SYS>>Dado un artículo de noticias, proporciona los siguientes campos en un diccionario JSON: 'titulo', 'SEO' y 'resumen'<</SYS>> "La Revolución Industrial, que tuvo lugar principalmente en el siglo XIX, fue un período de grandes cambios tecnológicos, culturales y socioeconómicos que transformó a las sociedades agrarias en sociedades industriales. Durante este tiempo, hubo un cambio masivo de mano de obra de las granjas a las fábricas. Esto se debió a la invención de nuevas máquinas que podían realizar tareas más rápido y eficientemente que los humanos o los animales. Esta transición llevó a un aumento en la producción de bienes, pero también tuvo consecuencias negativas, como la explotación \laboral y la contaminación ambiental. [/INST]

  Sure! Here is the requested JSON dictionary for the article:

{
"titulo": "La Revolución Industrial",
"SEO": "Revolución Industrial: Un período de cambios tecnológicos, culturales y socioeconómicos que transformaron a las sociedades agrar

## 2. Selección y preparación del conjunto de datos

Al igual que para el resto de los LLMs, uno de los puntos más importantes de la selección y preparación de los datos es el formato de los ejemplos de entrenamiento.

En el caso de Llama 2, los autores utilizaron la siguiente plantilla para los modelos de chat:
```
<s>[INST] <<SYS>>
System prompt
<</SYS>>

User prompt [/INST] Model answer </s>
```
La plantilla se compone de elementos opcionales y obligatorios:
* Instrucciones del sistema (opcionales) para guiar al modelo
* Instrucciones del usuario (obligatorias) para dar la instrucción
* Entradas adicionales (opcionales) a tener en cuenta
* Respuesta del modelo (obligatoria)

A continuación puede observarse un ejemplo de nuestro conjunto de datos:

```
<s>[INST] <<SYS>>Dado un artículo de noticias, proporciona los siguientes campos en un diccionario JSON: \'titulo\', \'SEO\' y \'resumen\'<</SYS>>  La empresa pública Canal de Isabel II congelará este 2019 las tarifas de agua por cuarto año consecutivo para los hogares de la región. Además, el Canal ha eliminado las órdenes de corte de suministro por impago a aquellas familias que no puedan abonar el suministro por problemas económicos, y extenderá bonificaciones a los perceptores de pensiones de viudedad con ingresos inferiores a los 14.000 euros brutos anuales. El bono social de Canal se aplica ya a los perceptores de la Renta Mínima de Inserción y de la Renta Activa de Inserción del Servicio Público de Empleo Estatal, pensiones no contributivas y, en general, a todas aquellas personas en situación de vulnerabilidad, además de familias numerosas. La Comunidad de Madrid volverá a congelar las tarifas del transporte público para este año 2019. Según lo recogido en los presupuestos regionales, los precios del abono transporte y los billetes sencillos para Metro, EMT y autobuses interurbanos se mantendrán en los mismos niveles que se fijaron en 2013. Se mantiene también la tarifa plana de 20 euros para el Abono de Transporte Joven, que ya cuenta con 1,3 millones de usuarios. Aparte, para este año se prevé la apertura de la nueva estación de Metro de Arroyo Fresno (L7) para el primer trimestre del año y que dará servicio a 220.000 vecinos de los barrios periféricos del noroeste de la capital. A ello se sumará también la reapertura de la \\"renovada\\" estación de Gran Vía, que servirá de intercambiador con la estación de Cercanías de la estación de Sol. También continuarán los trabajos para ampliar la Línea 11. A nivel estatal, el año 2019 se inicia con subidas en la luz, los carburantes, varios productos de telefonía, los sellos y los billetes regionales, mientras que bajará el gas natural, en una media del 1,92% el IBI y se mantendrán las tarifas de Cercanías y las tasas aeroportuarias [Qué sube y qué baja en 2019]. Sigue con nosotros la actualidad de Madrid en Facebook, en Twitter y en nuestro Patio de Vecinos en Instagram[/INST] "{\\"titulo\\": \\"Congelación de tarifas de agua y transporte público en Madrid para 2019\\", \\"SEO\\": [\\"Canal de Isabel II, tarifas de agua, transporte público, Madrid, 2019\\"], \\"resumen\\": \\"Canal de Isabel II congela tarifas de agua. Madrid mantiene precios de transporte público y anuncia mejoras en 2019.\\"}" </s>```



### 2.1. Lectura del conjunto de datos

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Leemos el conjunto de datos de Google Drive
dataset_path = "/content/drive/MyDrive/datasets/llama-2-fine-tuning-datset.txt"

with open(dataset_path, 'r', encoding='utf-8') as f:
  examples = f.read().splitlines() # De esta forma no sale el \n

In [18]:
from datasets import Dataset, DatasetDict

ds = Dataset.from_dict({"text": examples})

In [19]:
ds

Dataset({
    features: ['text'],
    num_rows: 10
})

In [20]:
ds["text"][2]

'<s>[INST] <<SYS>>Dado un artículo de noticias, proporciona los siguientes campos en un diccionario JSON: \'titulo\', \'SEO\' y \'resumen\'<</SYS>>  La empresa pública Canal de Isabel II congelará este 2019 las tarifas de agua por cuarto año consecutivo para los hogares de la región. Además, el Canal ha eliminado las órdenes de corte de suministro por impago a aquellas familias que no puedan abonar el suministro por problemas económicos, y extenderá bonificaciones a los perceptores de pensiones de viudedad con ingresos inferiores a los 14.000 euros brutos anuales. El bono social de Canal se aplica ya a los perceptores de la Renta Mínima de Inserción y de la Renta Activa de Inserción del Servicio Público de Empleo Estatal, pensiones no contributivas y, en general, a todas aquellas personas en situación de vulnerabilidad, además de familias numerosas. La Comunidad de Madrid volverá a congelar las tarifas del transporte público para este año 2019. Según lo recogido en los presupuestos reg

## 3. Fine-tuning del modelo

### 3.1. Configuración de LoRA

La siguiente función es interesante para comparar el número de parámetros entrenables que tiene el modelo antes y después de apalicar LoRA

In [21]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [22]:
print_trainable_parameters(model)

trainable params: 262410240 || all params: 3500412928 || trainable%: 7.50


Configuramos los parámetros de LoRA

In [23]:
from peft import LoraConfig, get_peft_model, prepare_model_for_int8_training

# Definición de la configuración de LoRA
lora_config = LoraConfig(
                 r = 16, # Dimensión de las matrices
                 lora_alpha = 16, # LoRA scaling factor
                 lora_dropout = 0.05, # Regularización
                 bias="none",
                 task_type="CAUSAL_LM" # Tipo de tarea/modelo al que aplicarlo
                 # target_modules=["q", "k", "v"], # Los módulos (ej. Attention Heads) donde aplicar las matrices
)

In [24]:
# Aplicamos la configuración al modelo
model_peft = get_peft_model(model, lora_config)

# Mostramos el número de parámetros que se van a entrenar
model_peft.print_trainable_parameters()

trainable params: 8,388,608 || all params: 3,508,801,536 || trainable%: 0.23907331075678143


### 3.2. Preparación y ejecución del fine-tuning (entrenamiento)

### Definición de los parámetros del entrenamiento

In [25]:
# Directorio de salida donde se almacenarán las predicciones del modelo y los puntos de control
output_dir = "/content/drive/MyDrive/llama2-7b-chat-peft"

# Número de epochs de entrenamiento
num_train_epochs = 5

# Habilitar entrenamiento fp16/bf16 (establecer bf16 en True con un A100)
fp16 = False
bf16 = False

# Tamaño del lote por GPU para el entrenamiento
per_device_train_batch_size = 4

# Tamaño del lote por GPU para la evaluación
per_device_eval_batch_size = 4

# Número de pasos de actualización para acumular los gradientes
gradient_accumulation_steps = 1

# Habilitar el registro de puntos de control de gradientes
gradient_checkpointing = True

# Norma máxima del gradiente (recorte del gradiente)
max_grad_norm = 0.3

# Tasa de aprendizaje inicial (optimizador AdamW)
learning_rate = 2e-4

# Decaimiento de pesos a aplicar a todas las capas excepto a los pesos de sesgo/LayerNorm
weight_decay = 0.001

# Optimizador a usar
optim = "paged_adamw_32bit"

# Programación de la tasa de aprendizaje
lr_scheduler_type = "cosine"

# Número de pasos de entrenamiento (anula num_train_epochs)
max_steps = -1

# Proporción de pasos para un calentamiento lineal (de 0 a tasa de aprendizaje)
warmup_ratio = 0.03

# Agrupar secuencias en lotes del mismo tamaño
# Ahorra memoria y acelera considerablemente el entrenamiento
group_by_length = True

# Guardar punto de control cada X pasos de actualización
save_steps = 0

# Registrar cada X pasos de actualización
logging_steps = 25

In [26]:
from transformers import TrainingArguments
from trl import SFTTrainer

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    remove_unused_columns=False,
    save_strategy="epoch",
    save_total_limit=2
)

# Creamos la instancia de entrenamiento
trainer = SFTTrainer(
    model=model,
    train_dataset=ds,
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=None, # Cuando es None, el max_seq_len vendrá determinado por la secuencia más larga de un lote
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False, # Empaquetar múltiples ejemplos cortos en la misma secuencia de entrada para aumentar la eficiencia
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

### 3.3. Entrenamiento y almacenamiento del modelo

In [27]:
# Iniciamos el entrenamiento
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning

TrainOutput(global_step=15, training_loss=2.3633514404296876, metrics={'train_runtime': 186.7374, 'train_samples_per_second': 0.268, 'train_steps_per_second': 0.08, 'total_flos': 1036908923879424.0, 'train_loss': 2.3633514404296876, 'epoch': 5.0})

In [28]:
# Guardamos el tokenizador en disco para utilizarlo posteriormente
tokenizer.save_pretrained(f"{output_dir}/tokenizer")

('/content/drive/MyDrive/llama2-7b-chat-peft/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/llama2-7b-chat-peft/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/llama2-7b-chat-peft/tokenizer/tokenizer.model',
 '/content/drive/MyDrive/llama2-7b-chat-peft/tokenizer/added_tokens.json',
 '/content/drive/MyDrive/llama2-7b-chat-peft/tokenizer/tokenizer.json')

## 4. Generación de texto con LLAMA 2 Fine-tuned

### 4.1. Lectura del modelo y del tokenizador

In [29]:
from transformers import pipeline

# Creamos un pipeline para la tokenización y generación del texto
llama2_ft_pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)

In [30]:
prompt = """"La Revolución Industrial, que tuvo lugar principalmente en el siglo XIX, \
fue un período de grandes cambios tecnológicos, culturales y socioeconómicos que \
transformó a las sociedades agrarias en sociedades industriales. Durante este tiempo, \
hubo un cambio masivo de mano de obra de las granjas a las fábricas. Esto se debió a \
la invención de nuevas máquinas que podían realizar tareas más rápido y eficientemente \
que los humanos o los animales. Esta transición llevó a un aumento en la producción de \
bienes, pero también tuvo consecuencias negativas, como la explotación \laboral y la \
contaminación ambiental."""

In [31]:
prompt_template = f"<s>[INST] <<SYS>>Dado un artículo de noticias, proporciona \
los siguientes campos en un diccionario JSON: \'titulo\', \'SEO\' y \'resumen\'\
<</SYS>> {prompt} [/INST]"

# Invocamos el pipeline para realizar generación de texto
output = llama2_ft_pipe(prompt_template)
print(output[0]['generated_text'].replace("[/INST]", "[/INST]\n\n"))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:91: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>[INST] <<SYS>>Dado un artículo de noticias, proporciona los siguientes campos en un diccionario JSON: 'titulo', 'SEO' y 'resumen'<</SYS>> "La Revolución Industrial, que tuvo lugar principalmente en el siglo XIX, fue un período de grandes cambios tecnológicos, culturales y socioeconómicos que transformó a las sociedades agrarias en sociedades industriales. Durante este tiempo, hubo un cambio masivo de mano de obra de las granjas a las fábricas. Esto se debió a la invención de nuevas máquinas que podían realizar tareas más rápido y eficientemente que los humanos o los animales. Esta transición llevó a un aumento en la producción de bienes, pero también tuvo consecuencias negativas, como la explotación \laboral y la contaminación ambiental. [/INST]

  Sure! Here is the requested JSON dictionary for the article:

{
"titulo": "La Revolución Industrial",
"SEO": ["Revolución Industrial", "Industrial Revolution", "historia", "cultura", "sociedad", "economía"],
"resumen": "La Revolución Indu

### 4.2. Cargar el modelo almacenado en disco

In [35]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
!pip install sentencepiece

In [4]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM

model_name = "NousResearch/Llama-2-7b-chat-hf"
adapters_name = "/content/drive/MyDrive/llama2-7b-chat-peft/checkpoint-15"

print(f"Cargando el modelo: '{model_name}' en memoria...")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)

model = PeftModel.from_pretrained(model, adapters_name)
model = model.merge_and_unload()

print(f"El modelo: '{model_name}' ha sido cargado correctamente")

Cargando el modelo: 'NousResearch/Llama-2-7b-chat-hf' en memoria...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

El modelo: 'NousResearch/Llama-2-7b-chat-hf' ha sido cargado correctamente


In [5]:
from transformers import AutoTokenizer

# Leemos el tokenizador
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/llama2-7b-chat-peft/tokenizer", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [6]:
from transformers import pipeline

# Creamos un pipeline para la tokenización y generación del texto
llama2_ft_pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)

In [7]:
prompt = """"La Revolución Industrial, que tuvo lugar principalmente en el siglo XIX, \
fue un período de grandes cambios tecnológicos, culturales y socioeconómicos que \
transformó a las sociedades agrarias en sociedades industriales. Durante este tiempo, \
hubo un cambio masivo de mano de obra de las granjas a las fábricas. Esto se debió a \
la invención de nuevas máquinas que podían realizar tareas más rápido y eficientemente \
que los humanos o los animales. Esta transición llevó a un aumento en la producción de \
bienes, pero también tuvo consecuencias negativas, como la explotación \laboral y la \
contaminación ambiental."""

In [8]:
prompt_template = f"<s>[INST] <<SYS>>Dado un artículo de noticias, proporciona \
los siguientes campos en un diccionario JSON: \'titulo\', \'SEO\' y \'resumen\'\
<</SYS>> {prompt} [/INST]"

# Invocamos el pipeline para realizar generación de texto
output = llama2_ft_pipe(prompt_template)
print(output[0]['generated_text'].replace("[/INST]", "[/INST]\n\n"))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>[INST] <<SYS>>Dado un artículo de noticias, proporciona los siguientes campos en un diccionario JSON: 'titulo', 'SEO' y 'resumen'<</SYS>> "La Revolución Industrial, que tuvo lugar principalmente en el siglo XIX, fue un período de grandes cambios tecnológicos, culturales y socioeconómicos que transformó a las sociedades agrarias en sociedades industriales. Durante este tiempo, hubo un cambio masivo de mano de obra de las granjas a las fábricas. Esto se debió a la invención de nuevas máquinas que podían realizar tareas más rápido y eficientemente que los humanos o los animales. Esta transición llevó a un aumento en la producción de bienes, pero también tuvo consecuencias negativas, como la explotación \laboral y la contaminación ambiental. [/INST]

  ¡Claro! A continuación, te proporciono los campos solicitados en un diccionario JSON para el artículo de noticias que proporcionaste:

{
"titulo": "La Revolución Industrial",
"SEO": ["Revolución Industrial", "s. XIX", "transición agraria-